# Choosing the right neuron model

In [ ]:
import sinabs
import sinabs.layers as sl
import torch
import matplotlib.pyplot as plt

Let's start by creating a helpful plotting function and some constant current input.

In [ ]:
def plot_evolution(neuron_model: sinabs.layers, input: torch.Tensor):
    time_steps = input.shape[1]

    neuron_model.reset_states()
    v_mem = []
    spikes = []
    for step in range(time_steps):
        output = neuron_model(input[:, step])
        v_mem.append(neuron_model.v_mem.detach().numpy())
        spikes.append(output.sum().detach().numpy())
    
    plt.plot(v_mem, label='v_mem')
    plt.plot(spikes, label='spike activation', drawstyle='steps', linewidth=2)
    if neuron_model.activation_fn:
        plt.plot([neuron_model.activation_fn.spike_threshold] * time_steps, '--', label='spike threshold', )
    plt.xlabel('time')
    plt.title(f"{neuron_model.__class__.__name__} neuron dynamics")
    plt.legend()

input = torch.ones((1, 300, 1)) * 0.03

## Integrate and Fire neuron
This neuron has no leakage and simply integrates all the input it receives.

In [ ]:
iaf_neuron = sl.IAF()
plot_evolution(iaf_neuron, input)

## Leaky Integrate and Fire neuron
This neuron integrates the input and decays its state at every time step. It emits a spike whenever the membrane potential is above the spike threshold

In [ ]:
lif_neuron = sl.LIF(tau_mem=40., norm_input=False)
plot_evolution(lif_neuron, input)

## Leaky integrator neuron
Same as LIF, just without activation function.

In [ ]:
exp_leak_neuron = sl.ExpLeak(tau_leak=40.)
plot_evolution(exp_leak_neuron, input)

## Adaptive Leaky Integrate and Fire neuron
This is a LIF neuron with an adaptive threshold.

In [ ]:
alif_neuron = sl.ALIF(tau_mem=40., tau_adapt=10.)
plot_evolution(alif_neuron, input)